# Basic Ideas of Finding Regions
Detailed exploration of the method proposed in [this paper](https://proceedings.mlr.press/v221/liu23a.html) by Liu et al. but focused on a single-layer approach.

## Simulating a ReLU layer

We start by simulating a ReLU layer from $\mathbb{R}^2\to\mathbb{R}^3$. This require a $(3\times2)$ weight matrix $W$ and a $(3\times1)$ bias vector $b$:

In [1]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

W = np.random.randn(3,2)
b = np.random.randn(3,1)
print("W:", W)
print("b:", b)

W: [[ 0.49671415 -0.1382643 ]
 [ 0.64768854  1.52302986]
 [-0.23415337 -0.23413696]]
b: [[ 1.57921282]
 [ 0.76743473]
 [-0.46947439]]


We also need a random point in $\mathbb{R}^2$ and a ReLU function, for now we implement the ReLU function and its derivative brute force as:

\begin{equation*}
    \text{ReLU}(a) = \begin{cases}
        a \; \text{if}\; a>0 \\
        0 \; \text{if}\; a\leq 0.
    \end{cases}
\end{equation*}

\begin{equation*}
    \text{dReLU}(a) = \begin{cases}
        1 \; \text{if}\; a>0 \\
        0 \; \text{if}\; a\leq 0.
    \end{cases}
\end{equation*}

In [2]:
def ReLU(a):
    return np.maximum(a, 0)

def dReLU(a):
    return np.where(a > 0, 1, 0)

# Random point in R^2
x = np.random.randn(2,1)

Next, we compute the activation $a=Wx+b$ of an input $x$, where $a\in\mathbb{R}^3$, the layer output $z=\text{ReLU}(a)$ and the derivative $q=\text{dReLU}(a)$:

In [3]:
activation = W @ x + b
print("Activation a:\n", activation)
z = ReLU(activation)
print("Output z:\n", z)
q = dReLU(activation)
print("Gradient q:\n", q)

Activation a:
 [[ 1.91278419]
 [ 0.41304567]
 [-0.48801344]]
Output z:
 [[1.91278419]
 [0.41304567]
 [0.        ]]
Gradient q:
 [[1]
 [1]
 [0]]


## Linear Model for gradients

The gradients of ReLU layers are binary vector with entries of $0$ and $1$. This can be turned into a sign vector with entries $-1$ and $1$:

\begin{equation*}
    \text{SIGN}(a) = -2q(a) + 1
\end{equation*}

This will map a vector $(1,0)\to(-1,1)$

In [15]:
def SIGN_from_activation(a):
    return -2 * dReLU(a) + 1
def SIGN_from_gradient(grad):
    return np.where(grad > 0, -1, 1)

assert all(SIGN_from_activation(activation) == SIGN_from_gradient(dReLU(activation)))
sign_vector = SIGN_from_activation(activation)
print("Sign vector:\n", sign_vector)

Sign vector:
 [[-1]
 [-1]
 [ 1]]


This is useful because any datapoint $x$ with gradient $q$ and corresponding sign vector $s'$ satisfies the following linearity:

\begin{equation*}
    \text{diag}(s')(Wx+b) \leq 0
\end{equation*}

Expanding this to:

\begin{align*}
    A &= \text{diag}(s')W \\
    c &= -\text{diag}(s')b
\end{align*}

enables us to write this in a more compact way as

\begin{equation*}
    Ax\leq c
\end{equation*}.

So, each gradient vector $q$ gives rise to a convex regions, described by the solution of the linear program above. 

In [ ]:
Amat = np.diag(sign_vector.flatten()) @ W 
cmat = -np.diag(sign_vector.flatten()) @ b

print("Amat:\n", Amat)
print("Cmat:\n", cmat)

Amat:
 [[-0.49671415  0.1382643 ]
 [-0.64768854 -1.52302986]
 [-0.23415337 -0.23413696]]
Cmat:
 [[1.57921282]
 [0.76743473]
 [0.46947439]]


## Feasibility

The next step is to determine of the region is feasible at all. 

In [24]:
import torch
def check_feasibility_torch(A, c, max_iters=200, tol=1e-4, input_bound=None):
    if not isinstance(A, torch.Tensor):
        A = torch.tensor(A, dtype=torch.float32)
    if not isinstance(c, torch.Tensor):
        c = torch.tensor(c, dtype=torch.float32)

    m = A.shape[1]  # input dimension
    x = torch.randn((m, 1), requires_grad=True)
    optimizer = torch.optim.Adam([x], lr=0.05)

    for _ in range(max_iters):
        # Original inequality
        constraint_violations = A @ x - c
        
        if input_bound is not None:
            # Bounding box constraints: -bound <= x <= bound
            lower_bounds = torch.relu(-x - input_bound)
            upper_bounds = torch.relu(x - input_bound)
            
            loss = constraint_violations.sum() + lower_bounds.sum() + upper_bounds.sum()
        else:
            loss = torch.relu(constraint_violations).sum()
            
        if loss.item() < tol:
            return True
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return False

In [25]:
check_feasibility_torch(Amat,cmat)

True

In [32]:
# Repeat but for arbitrary bit vector:

def region_for_bit_vector(bit_vector):
    sign_vector = SIGN_from_gradient(np.array(bit_vector))
    Amat = np.diag(sign_vector.flatten()) @ W 
    cmat = -np.diag(sign_vector.flatten()) @ b

    return Amat, cmat if check_feasibility_torch(Amat, cmat) else None



In [30]:
import itertools
def generate_all_bit_vectors(bit_length):
    return list(itertools.product([0, 1], repeat=bit_length))

for bit_vector in generate_all_bit_vectors(3):
    region_for_bit_vector(bit_vector)

Feasibility for bit-vector (0, 0, 0): False
Feasibility for bit-vector (0, 0, 1): True
Feasibility for bit-vector (0, 1, 0): True
Feasibility for bit-vector (0, 1, 1): True
Feasibility for bit-vector (1, 0, 0): True
Feasibility for bit-vector (1, 0, 1): True
Feasibility for bit-vector (1, 1, 0): True
Feasibility for bit-vector (1, 1, 1): True


In [27]:


def vertices_from_H_enumeration(A, c, tol=1e-9):
    """
    Compute vertices of P = { x | A x <= c } in R^n by enumerating combinations of n constraints.
    A: (k, n) numpy array
    c: (k, 1) or (k,) numpy array
    Returns: (m, n) array of unique vertices (m may be zero)
    """
    A = np.asarray(A, dtype=float)
    c = np.asarray(c, dtype=float).reshape(-1)
    k, n = A.shape
    vertices = []

    # enumerate combinations of n rows
    for idxs in itertools.combinations(range(k), n):
        A_sub = A[list(idxs), :]
        # skip singular subsets
        if np.linalg.matrix_rank(A_sub) < n:
            continue
        c_sub = c[list(idxs)]
        try:
            x = np.linalg.solve(A_sub, c_sub)
        except np.linalg.LinAlgError:
            continue
        # feasibility test (allow small tolerance)
        if np.all(A.dot(x) <= c + 1e-8 + tol):
            vertices.append(np.round(x, 12))  # round to reduce numerical dupes

    if not vertices:
        return np.zeros((0, n))

    verts = np.unique(np.vstack(vertices), axis=0)
    return verts

In [ ]:
print(vertices_from_H_enumeration(Amat,cmat))

[[-2.92359839  0.91867617]
 [-2.61172976  0.60678568]]


## Multiple Layers

### Recursive weight matrices

They propose a recursive weight matrices and bias vectors for layer $l$: 

\begin{equation*}
    \hat{W}_l = W_l\text{diag}(q_{l-1})\hat{W}_{l-1}
\end{equation*}

and

\begin{equation}
    \hat{b}_l = W_l\text{diag}(q_{l-1})\hat{b}_{l-1}
\end{equation}
for $2\leq l \leq L$ and $\hat{W}_1=W_1$ and $\hat{b}_1 = b_1$

## Convex regions (polytopes/polyhedra)

The central idea is that points that share the same gradient $q$ belong to the same convex regions. We therefore define a convex polytope $\mathcal{P}$ in $\mathbb{R}^2$ (input space) as the set $\{ x'| q(x') = q(x) \forall x'\in\mathbb{R}^2 \}$